# Análsis de carpetas de investigación del 2023

Estos datos corresponden al periodo del 01/Enero/2023 al 31/Marzo/2023

In [17]:
import pandas as pd
import numpy as np

### Entendimiento de los datos

In [19]:
df = pd.read_csv('data/da_2023.csv')
df.shape

(64344, 22)

In [20]:
df.columns

Index(['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Año_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'alcaldia_hechos', 'municipio_hechos', 'colonia_datos',
       'fgj_colonia_registro', 'latitud', 'longitud'],
      dtype='object')

In [21]:
df.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,9361878,2023,Enero,2023-01-01,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,NaN,NaN,MORAL,OFENDIDO,...,Diciembre,2022-12-31,22:08:00,00:09:00,COYOACAN,NaN,INSURGENTES CUICUILCO,INSURGENTES CUICUILCO,19.305532,-99.186308
1,9361879,2023,Enero,2023-01-01,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,50.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,23:50:00,00:41:00,VENUSTIANO CARRANZA,NaN,MOCTEZUMA 2A SECCION IV,MOCTEZUMA 2A SECCIÓN,19.429797,-99.098672
2,9361880,2023,Enero,2023-01-01,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,34.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,20:30:00,00:44:00,GUSTAVO A. MADERO,NaN,SAN JUAN DE ARAGON 3A SECCION (U HAB) I,SAN JUAN DE ARAGÓN III SECCIÓN,19.451641,-99.076529
3,9361881,2023,Enero,2023-01-01,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,Masculino,NaN,FISICA,LESIONADO,...,Diciembre,2022-12-31,23:30:00,00:47:00,COYOACAN,NaN,XOTEPINGO,XOTEPINGO,19.323245,-99.138016
4,9361882,2023,Enero,2023-01-01,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,Masculino,90.0,FISICA,CADAVER,...,Diciembre,2022-12-31,10:30:00,00:56:00,GUSTAVO A. MADERO,NaN,NaN,NaN,NaN,NaN


In [10]:
df.dtypes

idCarpeta                 int64
Año_inicio                int64
Mes_inicio               object
FechaInicio              object
Delito                   object
Categoria                object
Sexo                     object
Edad                    float64
TipoPersona              object
CalidadJuridica          object
competencia              object
Año_hecho               float64
Mes_hecho                object
FechaHecho               object
HoraHecho                object
HoraInicio               object
alcaldia_hechos          object
municipio_hechos         object
colonia_datos            object
fgj_colonia_registro     object
latitud                 float64
longitud                float64
dtype: object

In [22]:
df.nunique()

idCarpeta               61333
Año_inicio                  1
Mes_inicio                  3
FechaInicio                90
Delito                    255
Categoria                  16
Sexo                        2
Edad                      100
TipoPersona                 2
CalidadJuridica             5
competencia                 3
Año_hecho                  39
Mes_hecho                  12
FechaHecho               1434
HoraHecho                1441
HoraInicio               1455
alcaldia_hechos            18
municipio_hechos           91
colonia_datos            1655
fgj_colonia_registro     1538
latitud                 57654
longitud                57653
dtype: int64

In [27]:
df["Año_hecho"].describe()

count    64337.000000
mean      2022.731616
std          1.271916
min       1962.000000
25%       2023.000000
50%       2023.000000
75%       2023.000000
max       2023.000000
Name: Año_hecho, dtype: float64

### Tenemos delitos cometidos desde el año **1962** a los cuáles se les abrió carpetas de investigación hasta el año **2023**

In [28]:
df["Año_hecho"].unique()

array([2022., 2023., 2021., 2018., 2019., 2020., 2013., 1998., 2007.,
       2017., 2014., 2015., 2010., 2011.,   nan, 2003., 2016., 2005.,
       2000., 2006., 1993., 1995., 2012., 2002., 2008., 1969., 1987.,
       2009., 1967., 1990., 1999., 2004., 1994., 1974., 1976., 1985.,
       1962., 1991., 1972., 2001.])

In [47]:
df["Categoria"].unique()

array(['DELITO DE BAJO IMPACTO',
       'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA',
       'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA', 'HECHO NO DELICTIVO',
       'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', 'VIOLACIÓN',
       'ROBO A NEGOCIO CON VIOLENCIA',
       'ROBO A REPARTIDOR CON Y SIN VIOLENCIA', 'HOMICIDIO DOLOSO',
       'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA',
       'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA',
       'ROBO A CASA HABITACIÓN CON VIOLENCIA',
       'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA', 'SECUESTRO'],
      dtype=object)

In [42]:
df.loc[df["Año_hecho"]==1962]

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
44757,9408996,2023,Marzo,2023-03-06,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,60.0,FISICA,VICTIMA Y DENUNCIANTE,...,Septiembre,1962-09-11,13:45:00,18:39:00,COYOACAN,NaN,DEL CARMEN,DEL CARMEN,19.351477,-99.158951


In [51]:
df["competencia"].unique()

array(['FUERO COMUN', 'HECHOS NO DELICTIVOS', 'INCOMPETENCIAS'],
      dtype=object)

## Resumen del entendimiento de los datos

1. Existen carpetas de investigación sobre delitos cometidos desde 1962, abiertas hasta el 2023.

2. Tenemos **16 categorías de delitos**.

3. Hay **5 tipos de calidades jurídicas**.

4. Describiendo los **3 competencias de delitos**:
    
    * Fuero común: Aquellos delitos que afectas directamente a la persona.
    * Hechos no delictivos: No son delitos.
    * Incompetencias: La persona es incapaz de ser juzgada o de ser responsable de sus acciones debido a condiciones físicas o mentales.
    
    
    
5. En la columna de alcaldías estamos considerando un **extra llamado 'fuera de la ciudad de méxico'**

6. Tenemos **255 tipos de delitos**.

---

#### Recomendaciones en el proceso de limpieza

- Eliminar las siguientes columnas:
    
        1. idCarpeta
        
        
- Renombrar las siguientes columnas:

        1. Año_hecho
        2. Año_inicio
        3.fgj_colonia_registro
        

- Cambiar el tipo de dato a la columna edades y al año_hecho

- Estandarizar los nombres de las columnas a minusculas